Con este programa calculamos percentiles por cada quality model y clasificamos las calidades

In [1]:
import os

home = os.path.expanduser("~")

db_name=  'test_1' #'test_new_creators_4'
input_dir = os.path.join(home, 'video_resources', 'image_datasets')
output_dir = os.path.join(home, 'video_resources', 'sql_database')
database_names=['enfsi','lfw','forenface'] # 'lfw', 'scface',  'enfsi', 'enfsi2015', 'xqlfw', 'utkface','chokepoint','forenface'
detector_names=['mtcnn'] # 'dlib', 'mtcnn', 'mtcnn_serfiq'
embedding_model_names=["QMagFace"] # , "Dlib", "ArcFace",
attributes_to_update=[]  # 'gender', 'age', 'emotion', 'race'
quality_model_names = ['ser_fiq', 'tface'] # 'confusion_score', 'ser_fiq',

In [2]:
from sql_face.sqldb import SQLDataBase

database = SQLDataBase(db_name=db_name,
                        input_dir = input_dir,
                        output_dir_name = output_dir,
                        database_names = database_names,
                        detector_names = detector_names,
                        embedding_model_names = embedding_model_names,
                        quality_model_names = quality_model_names)

2023-03-20 23:56:45.243881: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:


import pandas as pd
import numpy as np
from tqdm import tqdm
from sql_face.tables import EnfsiPair, EnfsiPair2015, QualityImage, FaceImage, CroppedImage, Image, QualityModel, QualityGroup

session = database.session

for qm in quality_model_names:

    sql = session.query(QualityImage).join(QualityModel).filter(QualityModel.name == qm).all()

    quality = [row.quality for row in sql if row.quality]



    percentiles = [20, 40, 60, 80, 99]
    

    quaperc = np.percentile(quality,percentiles)
    quaperc[-1] = np.inf

    n1 = 1000
    n = n1
    updated_qimages = []
    for row in tqdm(sql):

        quality_group = next(X_Enum   for X_Enum, umbral in zip(QualityGroup,quaperc) if row.quality < umbral)     
        
        updated_qimages.append({"qualityImage_id": row.qualityImage_id,"quality_group": quality_group})
            
        if not n:
            session.bulk_update_mappings(QualityImage, updated_qimages)
            session.commit()

            updated_qimages = []
            n = n1
        else:
            n-=1
    if updated_qimages:
        session.bulk_update_mappings(QualityImage, updated_qimages)
            

    session.commit() 

  0%|          | 1/863754 [00:32<7864:49:43, 32.78s/it]

In [ ]:
# query = (session.query(EnfsiPair).limit(100).all())

# for img in query:
#     filter = ['image_quality']
#     cat = img.get_category(im_category_list = filter,fi_cat_list=[],detector='mtcnn',embedding_model='Arcface')

#     cat2 = img.get_category([],[],'mtcnn','Arcface')

#     a = 5
